# Scraping, Cleaning and Optimization Codes

Libraries Used

In [1]:
import pandas as pd
from tqdm import tqdm
import urllib
import os
import time
from tqdm import tqdm
import requests
import json

**Loading Lamudi csv file from ML1 Project**

In [11]:
df = pd.read_csv('lamudi_clean.csv')
df.head()

,City,Amount,No_of_Bedroom,No_of_Baths,Floor_Area,Latitude,L0ngitude,countnewAm,floor_number
0,4,8500000.0,2.0,2.0,52.00,14.558112,120.993771,17,10.0
1,10,15700000.0,2.0,1.0,132.39,14.603980,121.045000,4,2.0
2,9,3300000.0,1.0,1.0,23.00,14.651189,121.032754,24,18.0
3,3,4500000.0,1.0,1.0,30.26,14.572573,121.048343,33,15.0
4,8,20645110.0,3.0,3.0,129.00,14.582640,121.078700,93,28.0


**Getting Only the Unique Latitude and Longitude from the CSV file**

In [12]:
df['Php/Area'] = df['Amount']/df['Floor_Area'] # adding a new columns to normalize the target which is Price/Area Value

In [13]:
df2=df[['Latitude','L0ngitude']].drop_duplicates()
df2

,Latitude,L0ngitude
0,14.558112,120.993771
1,14.603980,121.045000
2,14.651189,121.032754
3,14.572573,121.048343
4,14.582640,121.078700
...,...,...
6477,14.557854,121.053456
6497,14.550520,121.047380
6498,14.660110,121.032210
6502,14.683517,121.097078


### Scraping in Google Street API

In [49]:

headings = ['0','20','40','60','80','100','120','140','160',
            '180','200','220','240','260','280','300','320','340'] # headings are the angle of shot, we take a shot every 20 degrees


url = 'https://maps.googleapis.com/maps/api/streetview?size=640x640&location='
meta = 'https://maps.googleapis.com/maps/api/streetview/metadata?size=640x640&location='

key = #input google api key should be string
fov = '120' # this is the zoom in and zoom out option, 120 is the max zoom out 
source = 'outdoor' # an option to make sure the the image will be an outdoor shot
count = 0
pitch = '25' # this is the angle of elevation of the image/shot relative to the road
list1 = []
with tqdm(total=len(df2)*18) as pbar:

    for x, y in df2.iterrows():

        coordinates = str(y.tolist()[0])+','+str(y.tolist()[1])

        for x1 in headings:

            heading = x1

            metaurl = meta + coordinates + '&source=' + source + \
                '&heading=' + heading + '&pitch='+ pitch + '&fov=' + fov + '&key=' + key 

            finalurl = url + coordinates + '&source=' + source + \
                '&heading=' + heading + '&pitch='+ pitch+ '&fov=' + fov + '&key=' + key

            check = json.loads(requests.get(metaurl).text)['status']

            if check != 'ZERO_RESULTS':

                urllib.request.urlretrieve(finalurl, os.path.join(
                    'images/', str(x)+'_'+str(x1)+'.jpg'))

                list1.append(x)
            else:
                break
        
            pbar.update(1)
        time.sleep(1)
        

100%|██████████| 1620/1620 [23:43<00:00,  1.14it/s]


**Saving the List of Images,Names and Php/Area in to a target CSV File**

In [27]:
list2 = list(set(list1))
df3 = pd.DataFrame(columns=['Php/Area'])
df3['Php/Area']=list2
df3.to_csv('target.csv') ## will be used for the Main Notebook